# WSW Spike Charts

Iterate through folders, create charts for sessions affected by spikes.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Sessions have been pre-identified and placed into a seperate folder.

In [1]:
import os
import sys
import json
import re

from datetime import datetime

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Printable Class

Simple class that allows other classes to be printed.

In [2]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Image Gallery Library

Page + Gallery + Figure

In [3]:
from PIL import Image

class IndexPage(Printable):
    """Simple class to create HTML index page"""

    def __init__(self, fileName, details):

        self.fileName = fileName
        self.details = details

        self.galleries = {}
        
        # Determine relative path to "docs" folder
        docsPath = ""
        tmpPath = os.path.dirname(fileName)
        while os.path.basename(tmpPath) != "docs":
            if docsPath:
                docsPath += "/"
            docsPath += ".."
            tmpPath = os.path.dirname(tmpPath)           
        self.details["DOCS_PATH"] = docsPath

    
    def addGallery(self, galleryId, details):
        """Add a new gallery to the index page"""

        self.galleries[galleryId] = IndexGallery(details)
        
        return self.galleries[galleryId]


    def getHtml(self):
        """Get the final HTML using the templates"""

        template = os.path.join(projdir, "docs", "template", "main.template.html")
        with open(template, 'r') as f:
            html = f.read()

        for detail in self.details:
            html = html.replace("{" + f"{detail}" + "}", self.details[detail])

        galleriesHtml = ""
        for galleryId in self.galleries:
            galleriesHtml += self.galleries[galleryId].getHtml() + "\n"

        html = html.replace("{GALLERY_TEMPLATES}\n", galleriesHtml)

        return html


    def saveHtml(self):
        """Save the final HTML to disk"""

        html = self.getHtml()

        with open(self.fileName, 'w') as f:
            f.write(html)

        return html


class IndexGallery(Printable):
    """Simple class to create HTML gallery"""

    def __init__(self, details):

        self.details = details

        self.figures = {}

    
    def addFigure(self, figureId, relPath, fileName, details):
        """Add a new figure to the gallery"""

        self.figures[figureId] = IndexFigure(relPath, fileName, details)
        
        return self.figures[figureId]


    def getHtml(self):
        """Get the final HTML using the templates"""

        template = os.path.join(projdir, "docs", "template", "gallery.template.html")
        with open(template, 'r') as f:
            html = f.read()

        for detail in self.details:
            html = html.replace("{" + f"{detail}" + "}", self.details[detail])

        figuresHtml = ""
        for figureId in self.figures:
            figuresHtml += self.figures[figureId].getHtml() + "\n"

        html = html.replace("{FIGURE_TEMPLATES}\n", figuresHtml)

        return html


class IndexFigure(Printable):
    """Simple class to create HTML figure"""

    def __init__(self, relPath, fileName, details):

        self.relPath = relPath
        self.fileName = fileName
        self.details = details

        self.details["IMAGE_FILENAME"] = relPath + "/" + os.path.basename(fileName)


    def createThumb(self, suffix="-thumb"):
        """Create thumbnail for the figure"""

        root, ext = os.path.splitext(os.path.split(self.fileName)[1])
        thumbName = root + suffix + ext
        
        image = Image.open(self.fileName)
        width, height = image.size

        self.details["IMAGE_WIDTH"] = str(width)
        self.details["IMAGE_HEIGHT"] = str(height)

        thumb = image.resize((width // 4, height // 4), Image.Resampling.LANCZOS)
        thumb.save(os.path.join(os.path.dirname(self.fileName), thumbName))
        
        self.details["THUMB_FILENAME"] = os.path.join(self.relPath, thumbName)


    def getHtml(self):
        """Get the final HTML using the templates"""

        template = os.path.join(projdir, "docs", "template", "figure.template.html")
        with open(template, 'r') as f:
            html = f.read()

        for detail in self.details:
            html = html.replace("{" + f"{detail}" + "}", self.details[detail])

        return html

## Index Page

This is project specific

In [4]:
def saveIndexPage(filePath):
    """Save HTML page as index to images"""

    fileName = os.path.join(filePath, "index.html")
    
    # Friendly name - basename, minus extension, replacing underscores with spaces, inserting space after "surfing"
    friendlyName = os.path.basename(filePath)
    friendlyName = os.path.splitext(friendlyName)[0]
    friendlyName = friendlyName.replace('_', ' ')
    friendlyName = re.sub('(surfing)([0-9]+)',  '\\1 \\2', friendlyName)

    indexPageDetails = \
    {
        "TITLE_TEXT": friendlyName.split(' ')[0],
        "H1_TEXT": friendlyName,
        "H2_TEXT": "Spike Analysis",
        "CUSTOM_CSS": "css/custom.css",
        "FURTHER_HREF": "https://logiqx.github.io/gps-wizard/"
    }
    indexPage = IndexPage(fileName, indexPageDetails)
    
    galleryName = os.path.basename(filePath)
    galleryDetails = \
    {
        "P_TEXT": "Application of simple log filters"
    }
    gallery = indexPage.addGallery(galleryName, galleryDetails)

    # Search the image folder for PNG files
    for pngName in ['raw', 'on-fix', 'ehpe', 'ehve', 'hacc', 'sacc']:
        
        fileName = os.path.join(filePath, 'img', pngName + '.png')
        if os.path.exists(fileName):

                # TODO - improve image descriptions - ideally describing the filters
                figureDetails = \
                {
                    "ALT_TEXT": pngName,
                    "CAPTION_TEXT": pngName
                }
                figure = gallery.addFigure(pngName, "img", fileName, figureDetails)
                figure.createThumb(suffix="-thumb")

    html = indexPage.saveHtml()

## Chart Library

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

In [6]:
def setTitle(ax, title, y=1.01, fontsize=12):
    '''Set the title for the chart'''

    ax.set_title(title, y=y, fontsize=fontsize)


def plotDottedPoints(ax, xPoints, yPoints, color='silver', linestyle='dotted'):
    '''Plot all data points in dotted form, including non-contiguous points'''
    
    ax.plot(xPoints, yPoints, color=color, linestyle=linestyle)

    
def plotContiguousPoints(ax, xPoints, yPoints, interval, color='#1f77b4'):
    '''Plot groups of contiguous data points individually'''
    
    segmentIdx = -1
    prevTs = 0

    for i in range(len(xPoints)):
        currTs = xPoints[i]
        if i > 0:
            if ((currTs - prevTs) * 1000).astype(int) > interval:
                if segmentIdx >= 0:
                    ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')
                    segmentIdx = -1
            else:
                if segmentIdx < 0:
                    segmentIdx = i
        prevTs = currTs

    if segmentIdx >= 0:
        ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')


def setXAxis(ax, xPoints):
    '''Set up the x-axis'''
    
    # Determine the number of tick intervals
    duration = xPoints.max() - xPoints.min()
    tickInterval = 900   
    while duration // tickInterval > 12:
        tickInterval *= 2

    # Determine the first and last ticks
    firstTick = xPoints.min() // tickInterval * tickInterval
    if firstTick < xPoints.min():
        firstTick += tickInterval
    lastTick = xPoints.max() // tickInterval * tickInterval + 1
    
    # Define the ticks for the x-axis
    ax.set_xticks(np.arange(firstTick, lastTick, tickInterval))
    
    # Set the format of the ticks to HH:MM
    ax.xaxis.set_major_formatter(tck.FuncFormatter(lambda x, p: datetime.fromtimestamp(x).strftime('%H:%M')))
    
    
def setYAxis(ax, yPoints, ylabel, fontsize=11):
    '''Set up the y-axis'''

    # Ensure the y-axis starts at zero
    ax.set_ylim(ymin=0)

    # Ensure that number of satellites is displayed as an integer
    if np.issubdtype(yPoints.dtype, np.integer):
        ax.yaxis.set_major_locator(tck.MaxNLocator(integer=True))

    # Add commas for accuracy / error estimates that are extremely large
    if yPoints.max() > 1000:
        ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Add grid lines to maximise readability
    ax.grid(which='major', axis='y')

    # Set the y-axis label
    ax.set_ylabel(ylabel, fontsize=fontsize)


def createSubplot(ax, title, xPoints, yPoints, ylabel, interval):
    '''Create chart using the data provided'''

    setTitle(ax, title)

    plotDottedPoints(ax, xPoints, yPoints)
    plotContiguousPoints(ax, xPoints, yPoints, interval)

    setXAxis(ax, xPoints)
    setYAxis(ax, yPoints, ylabel)

In [7]:
def createFigure(filename, comment, numAxs):
    '''Create figure and subplots'''

    fig, axs = plt.subplots(numAxs, figsize=(16, 24), dpi=100)

    plt.subplots_adjust(hspace=0.3)

    plt.suptitle(f'{filename}', y=0.925, fontsize=18, fontweight='bold')
    
    fig.text(0.5, 0.905, comment, fontsize=14, fontweight='bold', horizontalalignment='center')

    return fig, axs


def saveFigure(fig, dirName, pngName, quiet):
    '''Save the image to disk'''
    
    # Ensure directory for PNG exists
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        
    # Save PNG
    fname = os.path.join(dirName, pngName + '.png')
    if not quiet:
        print(f"Saving {fname}...")
    else:
        print('.', end='')
    fig.savefig(fname, bbox_inches='tight', facecolor='w')

    # Ensure the charts do not appear in Jupyter
    plt.close(fig)


def closeFigure(fig):
    '''Close the figure to prevent it being displayed in Jupyter'''
    
    plt.close(fig)

    
def createImage(filePath, charts, data, filters, pngName, interval, quiet):
    '''Create charts for a GPS track'''

    # Determine the number of subplots
    numAxs = 0
    for chart in charts:
        if chart['field'] in data:
            numAxs += 1
    
    # Construct comment that describes the filters
    if filters:
        comment = 'Filtered by {}'.format(' and '.join(filters))
    else:
        comment = 'Unfiltered'

    # Create the figure and subplots
    filename = os.path.basename(filePath)
    fig, axs = createFigure(filename, comment, numAxs)
    if numAxs == 1:
        axs = [axs]

    # x-points are simply the timestamps
    xPoints = data['ts']
    
    # Generate each of the charts
    axsId = 0
    for i in range(len(charts)):
        chart = charts[i]

        if chart['field'] in data:
            ax = axs[axsId]
            title = chart['title']
            yPoints = data[chart['field']]
            label = chart['label']

            createSubplot(ax, title, xPoints, yPoints, label, interval)
            axsId += 1
        
    # Determine the target directory (sub-directory called "img") and save the figure
    dirName = os.path.join(filePath, 'img')
    saveFigure(fig, dirName, pngName, quiet)

    # Ensure the figure is not displayed in Jupyter
    closeFigure(fig)

In [8]:
PNG_RAW = 'raw'
PNG_ON_FIX = 'on-fix'

CHIPSET_UBX = 'UBX'
CHIPSET_SRF = 'SiRF'

def getScaledData(charts, data):
    '''Take a copy of the data and convert m/s to knots, where applicable'''

    scaledData = {}

    scaledData['ts'] = data['ts']
    if 'fix' in data:
        scaledData['fix'] = data['fix']
    
    for chart in charts:
        field = chart['field']

        if field in data:
            if 'scale' in chart:
                scaledData[field] = data[field] * chart['scale']
            else:
                scaledData[field] = data[field]

    return scaledData


def getInterval(data):
    '''Determine the typical interval (milliseconds) between timestamps'''

    tsDiffs = ((data['ts'][1:] - data['ts'][:-1]) * 1000).round().astype(int)
    uniqueValues, counts = np.unique(tsDiffs, return_counts=True)
    interval = uniqueValues[np.argmax(counts)]
    
    return interval


def getFilteredData(charts, unfilteredData, points):
    '''Get filtered data'''
    
    filteredData = {}

    filteredData['ts'] = unfilteredData['ts'][points].flatten()
    if 'fix' in unfilteredData:
        filteredData['fix'] = unfilteredData['fix'][points].flatten()

    for chart in charts:
        if chart['field'] in unfilteredData:
            filteredData[chart['field']] = unfilteredData[chart['field']][points].flatten()
        
    return filteredData


def createRawImage(filePath, charts, data, filters, interval, quiet):
    '''Create image showing the raw data'''

    createImage(filePath, charts, data, filters, PNG_RAW, interval, quiet)

    return data


def createOnFixImage(filePath, charts, chipset, data, filters, interval, config, quiet):
    '''Create image showing the on-fix data'''

    createImage(filePath, charts, data, filters, PNG_ON_FIX, interval, quiet)

    return data

        
def createEhpeImage(filePath, charts, chipset, data, filters, interval, config, quiet):
    '''Create image showing data after the EHPE filter'''

    if 'ehpe' in data:
        if chipset == CHIPSET_SRF:
            field = 'EHPE'
        elif chipset == CHIPSET_UBX:
            field = 'hAcc'
        else:
            field = 'EHPE'
        threshold = config.getNode(f'Filters/{chipset}/{field}')

        filters.append('{} <= {:.01f} m'.format(field, threshold))

        points = np.argwhere(data['ehpe'] <= threshold)
        if len(points) > 0:
            data = getFilteredData(charts, data, points)
            createImage(filePath, charts, data, filters, field.lower(), interval, quiet)
    
    return data


def createEhveImage(filePath, charts, chipset, data, filters, interval, config, quiet):
    '''Create image showing data after the EHVE filter'''

    if 'ehve' in data:
        if chipset == CHIPSET_SRF:
            field = 'SDOS'
        elif chipset == CHIPSET_UBX:
            field = 'sAcc'
        else:
            field = 'EHVE'
        threshold = config.getNode(f'Filters/{chipset}/{field}')

        filters.append('{} <= {:.01f} knots'.format(field, threshold))

        points = np.argwhere(data['ehve'] <= threshold)
        if len(points) > 0:
            data = getFilteredData(charts, data, points)
            createImage(filePath, charts, data, filters, field.lower(), interval, quiet)
    
    return data


def getChipset(ext, config):
    '''Determine GNSS chipset from file extension'''
    
    result = 'Default'
    
    chipsets = config.getNode('Chipsets')
    for chipset, extensions in chipsets.items():
        if ext in extensions:
            result = chipset            
    
    return result
 

def processTrack(inputPath, track, config, quiet):
    '''Process a GPS track'''
    
    # Determine the relative path of the input file
    inputSubDir = config.getInputSubDir()
    relPath = inputPath.replace(f'{inputSubDir}/', '')

    # Determine absolute output path
    outputSubDir = config.getOutputSubDir()
    filePath = os.path.join(outputSubDir, relPath)
    
    # Determine the GNSS chipset
    ext = os.path.splitext(filePath)[1].lower()
    chipset = getChipset(ext, config)

    # Determine the group of charts based on the GNSS chipset
    charts = config.getNode(f'Charts/{chipset}')

    # Initialise the data
    rawData = getScaledData(charts, track.data)
    interval = getInterval(rawData)
    filters = []

    # Create raw image - i.e. no filters
    data = createRawImage(filePath, charts, rawData, filters, interval, quiet)

    # Filter data to on-fix
    if 'fix' in data:
        filters.append('fix > 0')
        points = np.argwhere(data['fix'] > 0)       
        if len(points) > 0:
            onFixData = getFilteredData(charts, rawData, points)
    else:
        onFixData = rawData
    onFixFilters = filters

    # Create EHPE image
    ehpeFilters = onFixFilters.copy()
    data = createEhpeImage(filePath, charts, chipset, onFixData, ehpeFilters, interval, config, quiet)

    # Create EHVE image
    ehveFilters = onFixFilters.copy()
    data = createEhveImage(filePath, charts, chipset, onFixData, ehveFilters, interval, config, quiet)
    
    # Create HTML gallery
    saveIndexPage(filePath)

## Main Function

In [9]:
class Config(Printable):
    """Simple config class"""

    def __init__(self, projdir, configFile):
        '''Create config object with JSON file pre-loaded'''
        
        self.projdir = projdir
        self.configFile = configFile

        configPath = os.path.join(projdir, configFile)
        
        with open(configPath) as f:
            jsonText = f.read()
            self.config = json.loads(jsonText)


    def getNode(self, nodePath):
        '''Get a specific node from the JSON'''
        
        node = self.config

        nodeNames = nodePath.split('/')       
        for nodeName in nodeNames:
            node = node[nodeName]
            
        return node


    def getInputSubDir(self):
        '''Get the input subdirectory as an absolute path'''
        
        subDir = os.path.join(self.projdir, self.getNode('Paths/InputDir'), self.getNode('Paths/SubDir'))
        
        return subDir


    def getOutputSubDir(self):
        '''Get the output subdirectory as an absolute path'''
        
        subDir = os.path.join(self.projdir, self.getNode('Paths/OutputDir'), self.getNode('Paths/SubDir'))
        
        return subDir

In [10]:
def processFile(filePath, config, quiet=False):
    '''Process a single GPS file'''

    reader = getFileReader(filePath)
    if not quiet:
        print(f"Loading {filePath}...")
    else:
        print('o', end='')

    # Some old UBX files from the ESP-GPS and the Luderitz devices contain bad checksums
    ext = os.path.splitext(filePath)[1].lower()
    if ext in ['.ubx']:
        reader.load(ignoreChecksums=True)
    else:
        reader.load()
    track = reader.tracks[0]
    
    processTrack(filePath, track, config, quiet)


def createIndex(filePath, config, files):
    '''Create simple index page for all files in the folder'''
    
    if files:
        if 'gpslogs' in filePath:
            filePath = filePath.replace('gpslogs', os.path.join('docs', 'gallery'))

        readme = os.path.join(filePath, 'README.md')

        with open(readme, 'w') as f:
            f.write('## GPS Spikes\n')
            f.write('### {}\n\n'.format(os.path.basename(filePath)))

            for file in sorted(files):
                f.write('- [{}]({}/index.html)\n'.format(file, file))
    

def processFiles(config):
    '''Iterate through session archive creating charts for each GPS file'''

    rootDir = config.getInputSubDir()
    extensions = [extension.lower() for extension in config.getNode('Extensions')]

    errors = {}
    
    for root, subDirs, files in os.walk(rootDir):
        processing = False
        processed = []
        
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in extensions:
                if not processing:
                    print("Processing {}...".format(root.replace(rootDir + '/', '')))
                    processing = True

                filePath = os.path.join(root, file)

                try:
                    processFile(filePath, config, quiet=True)
                    processed.append(file)
                        
                except Exception:
                    errors[filePath.replace(rootDir + '/', '')] = traceback.format_exc()
                    print('E', end='')

        createIndex(root, config, processed)

        if processing:
            print()

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

In [11]:
CONFIG_FILE = 'config/wsw_spike_charts.json'

if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    config = Config(projdir, CONFIG_FILE)

    processFiles(config)
    
    print(os.linesep + 'All done!')

Processing /home/jovyan/work/gps-wizard/../gps-spikes/gpslogs/motion/pos...
o...

All done!


## Adhoc Testing

In [12]:
if __name__ == '__main__':

    # ESP
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BN280E_84CCA86023F8_003.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BoomL_83AF2466E48_004.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_L_7C9EBDFAF5C8_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_R_7C9EBDFAF67C_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Boom_R_84CCA86023F8_004.ubx' # demonstrates hAcc filter
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/BoomL_83AF2466E48_008.ubx' # crazy acc values!
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_L_7C9EBDFAF5C8_007.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_R_7C9EBDFAF67C_006.ubx'
    
    #processFile(filePath)
    
    pass